**For educational purpose**

**Do not misuse!!**


Scraping https://www.sharesansar.com/ for price history as per your need

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     |████████████████████████████████| 958 kB 5.4 MB/s 
     |████████████████████████████████| 138 kB 49.9 MB/s 
     |████████████████████████████████| 356 kB 14.2 MB/s 
     |████████████████████████████████| 3.6 MB 42.9 MB/s 
     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 58 kB 5.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.down

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import csv

In [ ]:
# initializing browser support

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=chrome_options)

In [ ]:
BASE_URL = "https://www.sharesansar.com/company-list"

In [ ]:
driver.get(BASE_URL)

In [ ]:
dr1 = driver

In [ ]:

select = Select(dr1.find_element(By.NAME, "sector"))
# print(select.options)

In [ ]:

opts = []
i = 0
for opt in select.options:
  opts.append(opt.text)
  print(opt.text + '----->', i)
  i = i + 1

# selecting specific organization
sel = int(input("Which is the sector you would like to scrape: "))
select.select_by_index(sel)
dr1.find_element(By.ID, "btn_listed_submit").click()
print("You selected", opts[sel])

Commercial Bank-----> 0
Corporate Debentures-----> 1
Development Bank-----> 2
Finance-----> 3
Government Bonds-----> 4
Hotel & Tourism-----> 5
Hydropower-----> 6
Investment-----> 7
Life Insurance-----> 8
Manufacturing and Products-----> 9
Microfinance-----> 10
Mutual Fund-----> 11
Non-Life Insurance-----> 12
Others-----> 13
Preference Share-----> 14
Promoter Share-----> 15
Trading-----> 16
Which is the sector you would like to scrape: 0
You selected Commercial Bank


In [ ]:
time.sleep(5)
organizations = {}


while True:

  soup = BeautifulSoup(dr1.page_source)
  all_banks = soup.find("table", attrs = {"id": "myTable"})
  all_data = all_banks.find("tbody").find_all("tr")


  for row in all_data:
    table_data = row.find_all('td')
    key = f"{table_data[2].text} ({table_data[1].text})"
    val = table_data[1].find("a")["href"]

    organizations[key] = val

  try:
    elem = dr1.find_element(By.ID, "myTable_next")
    check = elem.get_attribute("class").split(" ")[-1]
    if check == "disabled":
      break
    dr1.find_element(By.ID, "myTable_next").click()
    time.sleep(5)
  except:
    break
  
print(f"we have {len(organizations)} organizations")
time.sleep(5)

we have 26 organizations


In [ ]:
print(f"which {opts[sel]} organization do you like to scrape")
i = 0
names = []
for key in organizations:
  print(key+"--->", i)
  names.append(key)
  i = i + 1

sel = int(input("enter reference number: "))
print("you selected ", names[sel])
key = names[sel]
# print(key)

which Commercial Bank organization do you like to scrape
NMB Bank Limited (NMB)---> 0
Siddhartha Bank Limited (SBL)---> 1
Nepal Credit & Commerce Bank Limited (NCCB)---> 2
Kumari Bank Limited (KBL)---> 3
Laxmi Bank Limited (LBL)---> 4
Machhapuchchhre Bank Limited (MBL)---> 5
Everest Bank Limited (EBL)---> 6
Nepal Bangladesh Bank Limited (NBB)---> 7
Nepal SBI Bank Limited (SBI)---> 8
Himalayan Bank Limited (HBL)---> 9
Standard Chartered Bank Nepal Limited (SCB)---> 10
Nepal Investment Bank Limited (NIB)---> 11
Nabil Bank Limited (NABIL)---> 12
Citizens Bank International Limited (CZBIL)---> 13
Prime Commercial Bank Limited (PCBL)---> 14
Sunrise Bank Limited (SRBL)---> 15
Agricultural Development Bank Limited (ADBL)---> 16
Sanima Bank Limited (SANIMA)---> 17
Mega Bank Nepal Limited (MEGA)---> 18
Civil Bank Limited (CBL)---> 19
Century Commercial Bank Limited (CCBL)---> 20
Nepal Bank Limited (NBL)---> 21
Global IME Bank Limited (GBIME)---> 22
NIC Asia Bank Limited (NICA)---> 23
Prabhu Ban

In [ ]:

dr1.get(organizations[key])


dr1.find_element(By.ID, "btn_cpricehistory").click()


select = Select(dr1.find_element(By.NAME, "myTableCPriceHistory_length"))
select.select_by_index(2)

run_once = 1
while True:

  time.sleep(2) 
  soup = BeautifulSoup(dr1.page_source)
  table = soup.find("table", attrs = {"id": "myTableCPriceHistory"})
  
  if run_once == 1:
    headings = table.find("thead").find_all("th")
    headings = [heading.text for heading in headings]
    with open(key+".csv", 'w', newline='') as output_file:
      dict_writer = csv.DictWriter(output_file, fieldnames = headings)
      dict_writer.writeheader() 
    run_once = 0

  table = table.find("tbody").find_all("tr")


  table_data = []
  for row in table:
    data = row.find_all("td")
    data = [d.text for d in data]
    table_data.append(dict(zip(headings, data)))
    
  print(f"found {len(table_data)} rows in this page")
  
  with open(key+".csv", 'a', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, fieldnames = headings)   
    dict_writer.writerows(table_data)

  try:
    dr1.find_element(By.ID, "myTableCPriceHistory_next").click()
    print("scraping next page")
  except:
    print("your work is done!!")
    break

found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scraping next page
found 50 rows in this page
scrapin